In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=088b399f153ff373fa5ad5e106fcb5c8af4b37dace655f09d13b192a8ecf0bbf
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Streaming").config('spark.ui.port', '4050').getOrCreate()

In [3]:
from pyspark.sql.types import *
schema = StructType([
StructField("person_ID",IntegerType(),True), StructField("name",StringType(),True), StructField("first",StringType(),True),
StructField("last", StringType(), True),
StructField("middle", StringType(), True),
StructField("email", StringType(), True),
StructField("phone", StringType(), True),
StructField("fax", StringType(), True),
StructField("title", StringType(), True)])

In [4]:
schema

StructType([StructField('person_ID', IntegerType(), True), StructField('name', StringType(), True), StructField('first', StringType(), True), StructField('last', StringType(), True), StructField('middle', StringType(), True), StructField('email', StringType(), True), StructField('phone', StringType(), True), StructField('fax', StringType(), True), StructField('title', StringType(), True)])

In [5]:
people_df = spark.readStream.format("csv").schema(schema).option("header", True).load("streaming")

In [6]:
print(people_df.isStreaming)

True


In [7]:
# this wont work
#people_df.show()

In [ ]:
results_df = people_df.select("*")
query = (results_df.writeStream
.format("json")
.outputMode("append")
.queryName("selectTable")
.option("checkpointLocation", "/content/checkpoint")
.option("path", "/content/results")
.start()
.awaitTermination() 
)